In [1]:
import os
import pandas as pd
import numpy as np
import gc

In [2]:
files = os.listdir('/home/web/files_for_research_Vova/')
files = [file for file in files if 'filtering' in file]

In [3]:
df = []
for c,i in enumerate(files):
    name = i.split('.')[0]
    tmp = pd.read_csv(os.path.join('/home/web/files_for_research_Vova/', i))
    tmp = tmp.rename(columns={'y_predicted_proba' : f'y_predict_proba_{name}'})
    if c!=0:
        tmp = tmp[[f'y_predict_proba_{name}', 'id']]
    tmp = tmp[tmp['id']!='id']
    df.append(tmp)
    
del tmp
gc.collect();
    

/home/web/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3221: DtypeWarning: Columns (0,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
overall_df = df[0]
for i in df[1:]:
    overall_df = overall_df.merge(i, left_index=True, right_index=True, how='inner')

In [8]:
overall_df['id'] = overall_df['id_x']
overall_df.drop(columns=[i for i in overall_df.columns \
                         if 'id' in i and len(i)>2], inplace=True)

# String to list dataframe

In [13]:
import ast 
def str_to_list(x):
    try:
        return list(ast.literal_eval(x))
    except:
        return None


def string_to_list_dataframe(df):
    columns = df.columns.tolist()
    columns_w_lists = []
    for column in columns:
        if df[column].astype(str). \
                apply(lambda x: x.startswith('[') and x.endswith(']')) \
                .astype(int).mean() > 0.8:
            columns_w_lists.append(column)
    for column in columns_w_lists:
        df[column] = df[column].apply(lambda x: str_to_list(x))
        df = df[~df[column].isna()]
    return df

In [14]:
overall_df = string_to_list_dataframe(overall_df)

# Get argmax on columns

In [15]:
cols_of_interest = [i for i in overall_df.columns.tolist() 
                   if 'y_predict_proba' in i]

In [16]:
for col in cols_of_interest:
    overall_df[col] = overall_df[col].apply(lambda x: np.argmax(x))

# Get samples where all model made mistake